In [1]:
import os
import json
from pathlib import Path

In [2]:
nltk_data_path = Path("~/.cache/nltk_data").expanduser()
nltk_data_path.mkdir(parents=True, exist_ok=True)
os.environ["NLTK_DATA"] = str(nltk_data_path)

In [3]:
# Deps for PDF parsing
!pip install "unstructured[pdf]"
!sudo apt-get install -y poppler-utils tesseract-ocr

# I can't even remember why we need this one
!pip install sentence-transformers

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.10 -m pip install --upgrade pip
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
poppler-utils is already the newest version (22.02.0-2ubuntu0.3).
0 upgraded, 0 newly installed, 0 to remove and 65 not upgraded.
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.10 -m pip install --upgrade pip


In [4]:
!pip install text_generation==0.6.1
!pip install langchain==0.0.354
!pip install langchain-community==0.0.8
!pip install langchain-core==0.1.5
!pip install langsmith==0.0.77

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.10 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.10 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.10 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.10 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.10 -m pip install

In [5]:
!pip install ipywidgets

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.10 -m pip install --upgrade pip


In [6]:
!pip uninstall faiss faiss-gpu -y
!pip install faiss-cpu

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.10 -m pip install --upgrade pip


In [7]:
import torch
import transformers
from langchain.schema import AIMessage, SystemMessage, HumanMessage

In [8]:
%load_ext autoreload
%autoreload 2
from util import (
    load_tokenizer,
    load_tgi_chat_model,
    HuggingFaceChatModel,
    VectorStoreRetrieverWithTextSplitter,
    parse_pdf,
    HuggingFaceChatModelWithBatchSupport
)

In [9]:
print(f"{transformers.__version__=}")
print(f"{torch.__version__=}")
print(f"{torch.version.cuda=}")
print(f"{torch.cuda.is_available()=}")
print(f"{torch.cuda.device_count()=}")

transformers.__version__='4.36.0'
torch.__version__='2.1.1+cu118'
torch.version.cuda='11.8'
torch.cuda.is_available()=True
torch.cuda.device_count()=1


In [10]:
!nvidia-smi

Fri Feb  9 18:51:10 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.154.05             Driver Version: 535.154.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3090        Off | 00000000:07:00.0 Off |                  N/A |
|  0%   42C    P8              11W / 420W |  19636MiB / 24576MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [11]:
!curl http://tgi:80/info

{"model_id":"TheBloke/OpenHermes-2.5-Mistral-7B-AWQ","model_sha":"a0e39e3a0ca6767307293819acb7c38e7a18cd31","model_dtype":"torch.float16","model_device_type":"cuda","model_pipeline_tag":"text-generation","max_concurrent_requests":128,"max_best_of":2,"max_stop_sequences":4,"max_input_length":4096,"max_total_tokens":6096,"waiting_served_ratio":1.2,"max_batch_total_tokens":86160,"max_waiting_tokens":20,"validation_workers":2,"version":"1.3.4","sha":"630800eed37b15c4b0c9eb8e6ab47212026720f7","docker_label":"sha-630800e"}

In [12]:
model_name='teknium/OpenHermes-2.5-Mistral-7B'
tgi_url = "http://tgi:80"

In [13]:
tokenizer = load_tokenizer(model_name)
chat_model = load_tgi_chat_model(
    tokenizer=tokenizer,
    inference_server_url=tgi_url,
    max_new_tokens=3000,
    timeout=900,
    do_sample=False,
    temperature=0.4,
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:149: UserWarning: Field "model_id" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [14]:
example = """\
Context:
----
Utility: These cards do something beneficial. For example, Call in a Favor lets you take resources of your choice from the supply.

Mandatory Quest: You can force an opponent to undertake a minor but pressing task before finishing other Quests. See “Mandatory Quests” on page 11 for more information.

2. Reassign Agent: After all Agents have been assigned in the round, each player with an Agent in Waterdeep Harbor reassigns that Agent to another action space. In effect, that player takes an additional turn with that Agent. Players reassign Agents in order, starting with the action space numbered “1.” You cannot reassign an Agent to Waterdeep Harbor.

Some game effects also allow players to reassign Agents during play.
----

Question: When can I assign agents?

Quotes:
----
After all Agents have been assigned in the round, each player with an Agent in Waterdeep Harbor reassigns that Agent to another action space.
----
----
Some game effects also allow players to reassign Agents during play.
----
"""

In [15]:
query = """\
Given a context and a question, quote the most relevant parts of the context that relate to the question. Do not attempt to answer the question, simply quote the relevant parts of the context. The following format will be used:

Context:
----
<document text here>
----

Question: <question text here>

Quotes:
----
<quote 1 here>
----
----
<quote 2 here>
----
----
<quote 3 here>
----

Now the actual context and question:

Context:
----
"ONE-SHOT” TREASURES
A Treasure card that says “Usable once only” is often called a “one-shot” Treasure. Most of these are used during combat to strengthen the munchkins or the monsters, and may be played from your hand or from the table. Some have other effects, however, so read the card carefully! Discard these cards as soon as the combat is over or their effect is resolved.
One-shot Items with a Gold Piece value may be sold for levels, just like other Items.

OTHER TREASURES
Other Treasure cards (like Go Up a Level cards) are not Items. Most of these cards say when they can be played, and whether they stay in play or are discarded. A couple of specific examples: Go Upa Level cards may be played on yourself or any other player at any time, even during combat. Discard them once they are played. Exception: You cannot play a Go Up a Level card to give a player the winning level!
Hireling may be played at any time, on any turn. You cannot give a Hireling an Item to carry while you are in combat, however.
----

Question: Can I play a Go Up a Level card during combat?

Quotes: \
"""

conversation = [
    SystemMessage(content="You are a helpful AI assistant that looks up board games rules relevant to any question asked."),
    HumanMessage(content=query),
]

chat_model.invoke(conversation)

AIMessage(content='"One-shot Items with a Gold Piece value may be sold for levels, just like other Items."\n\n"Go Upa Level cards may be played on yourself or any other player at any time, even during combat. Discard them once they are played. Exception: You cannot play a Go Up a Level card to give a player the winning level!"')

In [16]:
query = """\
Given a context and a question, quote the most relevant parts of the context that relate to the question. Do not attempt to answer the question, simply quote the relevant parts of the context verbatim. The following format will be used:

Question: <question text here>

Context:
----
<document text here>
----

Quotes:
----
<quote 1 here>
----
----
<quote 2 here>
----
----
<quote 3 here>
----

Example:

Question: When can I assign agents?

Context:
----
Utility: These cards do something beneficial. For example, Call in a Favor lets you take resources of your choice from the supply.

Mandatory Quest: You can force an opponent to undertake a minor but pressing task before finishing other Quests. See “Mandatory Quests” on page 11 for more information.

2. Reassign Agent: After all Agents have been assigned in the round, each player with an Agent in Waterdeep Harbor reassigns that Agent to another action space. In effect, that player takes an additional turn with that Agent. Players reassign Agents in order, starting with the action space numbered “1.” You cannot reassign an Agent to Waterdeep Harbor.

Some game effects also allow players to reassign Agents during play.
----

Quotes:
----
After all Agents have been assigned in the round, each player with an Agent in Waterdeep Harbor reassigns that Agent to another action space.
----
----
Some game effects also allow players to reassign Agents during play.
----

Now the actual context and question you are to answer:

Question: Can I play a Go Up a Level during combat?

Context:
----
When You May Take Actions

You may perform these actions at any time:
- Discard a Class or Race.
- Play a Go Up a Level or Hireling. Play a Curse.
You may perform these actions at any time, as long as you are not in combat: 
- Trade an Item with another player (the other player may not be in combat, either).
- Change which Items you have equipped.
- Play a card that you have just received (some cards may be played even during combat; see above).
You may perform these actions on your own turn:
- Play a new Class or Race card (at any time).
- Sell Items for levels (except when you are in combat).
- Play an Item (most Items cannot be played during combat, but some one-shot Items can; see p- 3).
----

Quotes:
----
"""

conversation = [
    SystemMessage(content="You are a helpful AI assistant that looks up board games rules relevant to any question asked."),
    HumanMessage(content=query),
]

print(chat_model.invoke(conversation).content)

You may perform these actions at any time, as long as you are not in combat: 
- Trade an Item with another player (the other player may not be in combat, either).
- Change which Items you have equipped.
- Play a card that you have just received (some cards may be played even during combat; see above).
You may perform these actions on your own turn:
- Play a new Class or Race card (at any time).
- Sell Items for levels (except when you are in combat).
- Play an Item (most Items cannot be played during combat, but some one-shot Items can; see p- 3).
----


In [17]:
from langchain_core.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough

prompt_template = """Given the following question and context, extract any part of the context *AS IS* that is relevant to answer the question. If none of the context is relevant return NO_OUTPUT. Do not prefix extracted parts with "-" or "*". 

Remember, *DO NOT* edit the extracted parts of the context.

> Question: {question}
> Context:
>>>
{context}
>>>
Extracted relevant parts:"""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["question", "context"],
)

question = "Can I play a Curse while in combat?"

context = """\
When You May Take Actions
You may perform these actions at any time: & Discard a Class or Race.
& Play a Go Up a Level or Hireling. Play a Curse.
You may perform these actions at any time, as long as you are not in combat: Trade an Item with another player (the other player may not be in combat, either).
y Change which Items you have equipped.
Play a card that you have just received (some cards may be played even during combat; see above).
You may perform these actions on your own turn: 3 Play a new Class or Race card (at any time).
we Sell Items for levels (except when you are in combat). Play an Item (most Items cannot be played during combat, but some one-shot Items can; see p- 3)."""

context2 = """\
COMBAT
To fight a monster, compare its combat strength to yours. Combat strength is the total of Level plus all modifiers - positive or negative - given by Items and other cards. If the monster's combat strength is equal to yours, or greater, you lose the combat and must Run Away (see p. 5). If your combat strength totals more than the monster's — note that monsters win ties! — you kill it and goupa level (two levels for some big monsters). You'll also get the number of Treasures shown on its card.
Sometimes a card will let you get rid of the monster without killing it. This is still “winning,” but you don't get a level. Unless the ability says otherwise, you don’t get the Treasures, either. If the last monster is removed from a combat, it ends instantly.
Some monster cards have special powers that affect combat — a bonus against a Race or Class, for instance. Be sure to check these!
You and the other players may play one-shot Treasures or use Class or Race abilities to help or harm you in your combat. Some Door cards may also be played into a combat, such as monster enhancers (see below).
If you kill a monster (or monsters!), discard the monster(s) and any other cards played, and claim your rewards. But note: someone may play a hostile card on you, or use a special power, just as you think you have won. When you kill a monster, you must wait a reasonable time, defined as about 2.6 seconds, for anyone else to speak up. After that, you have really killed the monster, and you really get the level(s) and Treasures, though they can still whine and argue.
"""

conversation = [
    # SystemMessage(content="You are a helpful AI assistant that looks up board games rules relevant to any question asked."),
    HumanMessage(content=query),
]

result = (prompt | chat_model).invoke({"question": question, "context": context})

print(context)
print()
print()
print("Extracted:")
print(result.content)

# print(chat_model.invoke(conversation).content)

When You May Take Actions
You may perform these actions at any time: & Discard a Class or Race.
& Play a Go Up a Level or Hireling. Play a Curse.
You may perform these actions at any time, as long as you are not in combat: Trade an Item with another player (the other player may not be in combat, either).
y Change which Items you have equipped.
Play a card that you have just received (some cards may be played even during combat; see above).
You may perform these actions on your own turn: 3 Play a new Class or Race card (at any time).
we Sell Items for levels (except when you are in combat). Play an Item (most Items cannot be played during combat, but some one-shot Items can; see p- 3).


Extracted:
You may perform these actions at any time, as long as you are not in combat: Trade an Item with another player (the other player may not be in combat, either).
y Change which Items you have equipped.
Play a card that you have just received (some cards may be played even during combat; see abo

In [18]:
from langchain_core.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
prompt_template = """Clearly restate the following excerpt from the board game Munchkin, without changing the meaning or the information conveyed. The rewritten text should be in the same language as the original text.
> Context:
>>>
{context}
>>>
Rewritten text:"""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["context"],
)

question = "Can I play a Curse while in combat?"

context = """\
When You May Take Actions
You may perform these actions at any time: & Discard a Class or Race.
& Play a Go Up a Level or Hireling. Play a Curse.
You may perform these actions at any time, as long as you are not in combat: Trade an Item with another player (the other player may not be in combat, either).
y Change which Items you have equipped.
Play a card that you have just received (some cards may be played even during combat; see above).
You may perform these actions on your own turn: 3 Play a new Class or Race card (at any time).
we Sell Items for levels (except when you are in combat). Play an Item (most Items cannot be played during combat, but some one-shot Items can; see p- 3)."""

context2 = """\
COMBAT
To fight a monster, compare its combat strength to yours. Combat strength is the total of Level plus all modifiers - positive or negative - given by Items and other cards. If the monster's combat strength is equal to yours, or greater, you lose the combat and must Run Away (see p. 5). If your combat strength totals more than the monster's — note that monsters win ties! — you kill it and goupa level (two levels for some big monsters). You'll also get the number of Treasures shown on its card.
Sometimes a card will let you get rid of the monster without killing it. This is still “winning,” but you don't get a level. Unless the ability says otherwise, you don’t get the Treasures, either. If the last monster is removed from a combat, it ends instantly.
Some monster cards have special powers that affect combat — a bonus against a Race or Class, for instance. Be sure to check these!
You and the other players may play one-shot Treasures or use Class or Race abilities to help or harm you in your combat. Some Door cards may also be played into a combat, such as monster enhancers (see below).
If you kill a monster (or monsters!), discard the monster(s) and any other cards played, and claim your rewards. But note: someone may play a hostile card on you, or use a special power, just as you think you have won. When you kill a monster, you must wait a reasonable time, defined as about 2.6 seconds, for anyone else to speak up. After that, you have really killed the monster, and you really get the level(s) and Treasures, though they can still whine and argue.
"""

conversation = [
    # SystemMessage(content="You are a helpful AI assistant that looks up board games rules relevant to any question asked."),
    HumanMessage(content=query),
]

result = (prompt | chat_model).invoke({"context": context})

print(context)
print()
print()
print("Rwritten:")
print(result.content)

# print(chat_model.invoke(conversation).content)

When You May Take Actions
You may perform these actions at any time: & Discard a Class or Race.
& Play a Go Up a Level or Hireling. Play a Curse.
You may perform these actions at any time, as long as you are not in combat: Trade an Item with another player (the other player may not be in combat, either).
y Change which Items you have equipped.
Play a card that you have just received (some cards may be played even during combat; see above).
You may perform these actions on your own turn: 3 Play a new Class or Race card (at any time).
we Sell Items for levels (except when you are in combat). Play an Item (most Items cannot be played during combat, but some one-shot Items can; see p- 3).


Rwritten:
At what times you may take actions: 

1. You are allowed to discard a Class or Race at any point.
2. You can play a Go Up a Level or Hireling card at any time, as well as a Curse card.
3. You can trade an Item with another player, as long as neither of you is in combat.
4. You can change the e

In [19]:
from langchain_core.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough

prompt_template = """Answer the following question based on the given context. Provide your reasoning first followed by the answer.

> Context:
>>>
{context}
>>>
> Question: {question}
> Answer: Let's think step by step."""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["question", "context"],
)

question = "Can I play a Curse while in combat?"

context = """\
When You May Take Actions
You may perform these actions at any time: & Discard a Class or Race.
& Play a Go Up a Level or Hireling. Play a Curse.
You may perform these actions at any time, as long as you are not in combat: Trade an Item with another player (the other player may not be in combat, either).
y Change which Items you have equipped.
Play a card that you have just received (some cards may be played even during combat; see above).
You may perform these actions on your own turn: 3 Play a new Class or Race card (at any time).
we Sell Items for levels (except when you are in combat). Play an Item (most Items cannot be played during combat, but some one-shot Items can; see p- 3)."""

context2 = """\
COMBAT
To fight a monster, compare its combat strength to yours. Combat strength is the total of Level plus all modifiers - positive or negative - given by Items and other cards. If the monster's combat strength is equal to yours, or greater, you lose the combat and must Run Away (see p. 5). If your combat strength totals more than the monster's — note that monsters win ties! — you kill it and goupa level (two levels for some big monsters). You'll also get the number of Treasures shown on its card.
Sometimes a card will let you get rid of the monster without killing it. This is still “winning,” but you don't get a level. Unless the ability says otherwise, you don’t get the Treasures, either. If the last monster is removed from a combat, it ends instantly.
Some monster cards have special powers that affect combat — a bonus against a Race or Class, for instance. Be sure to check these!
You and the other players may play one-shot Treasures or use Class or Race abilities to help or harm you in your combat. Some Door cards may also be played into a combat, such as monster enhancers (see below).
If you kill a monster (or monsters!), discard the monster(s) and any other cards played, and claim your rewards. But note: someone may play a hostile card on you, or use a special power, just as you think you have won. When you kill a monster, you must wait a reasonable time, defined as about 2.6 seconds, for anyone else to speak up. After that, you have really killed the monster, and you really get the level(s) and Treasures, though they can still whine and argue.
"""

rewritten_context = """\
At any time, you may perform the following actions: discard a class or race, play a go up a level or hireling, or play a curse. Additionally, when you are not in combat, you may also trade an item with another player (who must also not be in combat), change your equipped items, or play a card that you have just received (some cards can be played during combat, see above). On your turn, you may also perform the following actions: play a new class or race card, sell items for levels (except when in combat), and play an item (most items cannot be played during combat, but some one-shot items can; see page 3).
"""

rewrite_prompt_template = """Reword and improve clarity of the following excerpt from the board game Munchkin, without changing the meaning or the information conveyed. Use Markdown to format the text.

> Context:
>>>
{context}
>>>
Rewritten text:"""

rewrite_prompt = PromptTemplate(
    template=rewrite_prompt_template,
    input_variables=["context"],
)

context_to_use = (rewrite_prompt | chat_model).invoke({"context": context}).content
question = "Can I play a Go Up a Level card while in combat?"
result = (prompt | chat_model).invoke({"question": question, "context": context_to_use})

print(context_to_use)
print()
print()
print("Answer:")
print(result.content)

# print(chat_model.invoke(conversation).content)

You may take the following actions at any time:
- Discard a Class or Race.
- Play a Go Up a Level or Hireling.
- Play a Curse.

You may take these actions during non-combat situations:
- Trade an Item with another player (the other player must also not be in combat).
- Change which Items you have equipped.
- Play a card that you have just received (some cards can be played during combat, see above).

During your own turn, you may also:
- Play a new Class or Race card (at any time).
- Sell Items for levels (except when you are in combat).
- Play an Item (most Items cannot be played during combat, but some one-shot Items can; see p. 3).


Answer:
First, we need to identify the actions that can be taken during non-combat situations. According to the context, playing a Go Up a Level card is one of those actions. 

Second, we need to determine if the action can be taken during combat. The context states that "During your own turn, you may also: - Play a new Class or Race card (at any time).

In [20]:
from langchain_core.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate

system_prompt = """\
As an AI Assistant specialized in board game rules, your primary goal is to provide users with precise, understandable explanations and interpretations of game rules. Follow these key principles:

- **Base your responses on the official rulebooks or authoritative sources,** recognizing that these rules hold in all standard situations unless an explicit exception is stated. Avoid assumptions and unofficial variations unless specifically requested by the user.

- **Use clear, simple language** to explain rules. Avoid unnecessary jargon and ensure explanations are accessible to both new and experienced players.

- **Consider the gameplay context** in your interpretations, including game phases, player counts, and specific scenarios that might impact rule application.

- **Highlight rule variants and exceptions** clearly, explaining how they alter standard gameplay and under what circumstances they apply.

- **Attempt to understand user intent,** focusing on the aspect of the rule they might find confusing or the specific information they seek.

Your responses should empower users with a deeper understanding of the game rules, enhancing their gameplay experience.
"""

prompt_template = """Answer the following board game question based on the given context. Provide your step-by-step reasoning first followed by the answer. Each step should be a separate bullet point.

> Context:
>>>
{context}
>>>
> Question: {question}
> Answer: Let's think step by step."""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", prompt_template)
    ]
)

# prompt = PromptTemplate(
#     template=prompt_template,
#     input_variables=["question", "context"],
# )

context = """\
Items are the most common Treasures in Munchkin. They have a Gold Piece value, and a "No Value" card is equivalent to zero Gold Pieces. Items that are "carried" are considered to be in your possession, while those that are providing you with a bonus are "equipped." To indicate that an Item is not equipped, turn the card sideways. You cannot alter the status of your Items during combat or while running away.

Any player can carry any Item (except for extra Big items; see below), but you may equip only one Headgear, one suit of Armor, one pair of Footgear, and two "I Hand" Items (or one "2 Hands" Item) . . . unless you have a card that lets you ignore these limits, such as Hireling or Cheat!, or unless one of the cards says otherwise. If you are carrying two Headgear cards, for instance, you can equip only one of them at a time.

Some Items have restrictions, such as the Mace of Sharpness, which can only be wielded by a Cleric. Its bonus only counts for someone who is, at the moment, a Cleric.

You cannot discard Item cards "just because." You may sell Items for a level, trade Items with other players, or give an Item to another player who wants it (see below). You may discard Items to power certain Class and Race abilities. And a Curse or a monster's Bad Stuff (see p. 5) may force you to get rid of something!

Big Items: You may carry any number of Small items, but only one Big one. (Any item not marked Big is considered Small.) You may not discard one Big item to play another; you must sell the first Item, trade it, lose it to a Curse or Bad Stuff, or discard it to power a Class or Race ability."""


rewrite_prompt_template = """Reword and improve clarity of the following excerpt from the board game Munchkin, without changing the meaning or the information conveyed. Use Markdown to format the text.

> Context:
>>>
{context}
>>>
Rewritten text:"""

rewrite_prompt = PromptTemplate(
    template=rewrite_prompt_template,
    input_variables=["context"],
)

context_to_use = (rewrite_prompt | chat_model).invoke({"context": context}).content
question = "Can I carry multiple big items so long as only one is equipped?"
result = (prompt | chat_model).invoke({"question": question, "context": context_to_use})

print(context_to_use)
print()
print()
print("Answer:")
print(result.content)

# print(chat_model.invoke(conversation).content)

In Munchkin, items are the most common treasures and have a gold piece value. A "No Value" card is equivalent to zero gold pieces. Items that are "carried" are considered to be in your possession, while those that provide a bonus are "equipped." To indicate that an item is not equipped, turn the card sideways. You cannot alter the status of your items during combat or while running away.

Any player can carry any item (except for extra big items; see below), but you may equip only one headgear, one suit of armor, one pair of footgear, and two "I Hand" items (or one "2 Hands" item)... unless you have a card that lets you ignore these limits, such as Hireling or Cheat!, or unless one of the cards says otherwise. If you are carrying two headgear cards, for instance, you can equip only one of them at a time.

Some items have restrictions, such as the Mace of Sharpness, which can only be wielded by a Cleric. Its bonus only counts for someone who is, at the moment, a Cleric.

You cannot disc

In [21]:
why_wrong_prompt_template = """The following is a context, question, and incorrect answer. It is absolutely certain the answer is incorrect. Your job is to figure out why the answer is incorrect based on the context.

> Context:
>>>
{context}
>>>
> Question: {question}
> Answer: {answer}
> Explanation: Assuming the answer is wrong, why is it wrong?"""

why_wrong_prompt = PromptTemplate(
    template=why_wrong_prompt_template,
    input_variables=["question", "context", "answer"],
)

print((why_wrong_prompt | chat_model).invoke({
    "question": question,
    "answer": result.content,
    "context": context_to_use,
}).content)

The answer is wrong because it contradicts the given context. The context clearly states that you can carry any number of small items, but only one big one. The question asks if you can carry multiple big items as long as only one is equipped, but the context does not support this interpretation.


In [22]:
from langchain_core.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough

prompt_template = """Answer the following question based on the given context. Provide your reasoning first followed by the answer.

> Context:
>>>
{context}
>>>
> Question: {question}
> Answer: Let's think step by step."""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["question", "context"],
)

context = """\
Items are the most common Treasures in Munchkin. They have a Gold Piece value, and a "No Value" card is equivalent to zero Gold Pieces. Items that are "carried" are considered to be in your possession, while those that are providing you with a bonus are "equipped." To indicate that an Item is not equipped, turn the card sideways. You cannot alter the status of your Items during combat or while running away.

Any player can carry any Item (except for extra Big items; see below), but you may equip only one Headgear, one suit of Armor, one pair of Footgear, and two "I Hand" Items (or one "2 Hands" Item) . . . unless you have a card that lets you ignore these limits, such as Hireling or Cheat!, or unless one of the cards says otherwise. If you are carrying two Headgear cards, for instance, you can equip only one of them at a time.

Some Items have restrictions, such as the Mace of Sharpness, which can only be wielded by a Cleric. Its bonus only counts for someone who is, at the moment, a Cleric.

You cannot discard Item cards "just because." You may sell Items for a level, trade Items with other players, or give an Item to another player who wants it (see below). You may discard Items to power certain Class and Race abilities. And a Curse or a monster's Bad Stuff (see p. 5) may force you to get rid of something!

Big Items: You may carry any number of Small items, but only one Big one. (Any item not marked Big is considered Small.) You may not discard one Big item to play another; you must sell the first Item, trade it, lose it to a Curse or Bad Stuff, or discard it to power a Class or Race ability."""


rewrite_prompt_template = """Reword and improve clarity of the following excerpt from the board game Munchkin, without changing the meaning or the information conveyed. Use Markdown to format the text.

> Context:
>>>
{context}
>>>
Rewritten text:"""

rewrite_prompt = PromptTemplate(
    template=rewrite_prompt_template,
    input_variables=["context"],
)

context_to_use = (rewrite_prompt | chat_model).invoke({"context": context}).content
question = "Can I carry multiple big items so long as only one is equipped?"
result = (prompt | chat_model).invoke({"question": question, "context": context_to_use})

print(context_to_use)
print()
print()
print("Answer:")
print(result.content)

# print(chat_model.invoke(conversation).content)

# Items in Munchkin

Items are the most common Treasures in Munchkin and have a Gold Piece value. A "No Value" card is equivalent to zero Gold Pieces. Items that are "carried" are considered to be in your possession, while those that provide you with a bonus are "equipped." To indicate that an Item is not equipped, turn the card sideways. You cannot alter the status of your Items during combat or while running away.

Any player can carry any Item (except for extra Big items; see below), but you may equip only one Headgear, one suit of Armor, one pair of Footgear, and two "I Hand" Items (or one "2 Hands" Item) . . . unless you have a card that lets you ignore these limits, such as Hireling or Cheat!, or unless one of the cards says otherwise. If you are carrying two Headgear cards, for instance, you can equip only one of them at a time.

Some Items have restrictions, such as the Mace of Sharpness, which can only be wielded by a Cleric. Its bonus only counts for someone who is, at the mo

In [23]:
print("""I have generated the following responses to the question: Can I carry multiple big items so long as only one is equipped?\n\nResponse 0\nYou can carry multiple big items, but only one can be equipped at a time. The \"Big\" designation for big items does not affect the number of big items a player can carry, but it does affect which traps and curses affect them and prevents thieves from pocketing them.\n\nResponse 1\nAnswer: No, you cannot carry multiple Big items, even if only one is equipped. The rule states that you can carry any number of Small items and only one Big item.\n\nResponse 2\nAnswer: No, you cannot carry multiple Big items as long as only one is equipped. The rules state that you can equip only one Headgear, one suit of Armor, one pair of Footgear, and two \"I Hand\" Items (or one \"2 Hands\" Item) unless you have a card that allows you to ignore these limits, such as Hireling or Cheat!, or unless one of the cards says otherwise.\n\nResponse 3\nNo, you cannot carry multiple big items so long as only one is equipped. You can switch Big Items by selling, trading, losing to a Curse or Bad Stuff, or discarding for a Class or Race ability.\n\nResponse 4\nNo, you cannot carry multiple Big Items as long as only one is equipped. You can carry any number of Small items, but only one Big one.\n\nResponse 5\nAnswer: Based on the provided context, you can carry multiple big items, but you can only equip one at a time.\n\nResponse 6\nAnswer: No, you cannot carry multiple big items even if only one is equipped. You can only carry one big item at a time.\n\nEvaluate these responses.\nSelect the most consistent response based on majority consensus.\nStart your answer with \"The most consistent response is Response X\" (without quotes)""")

I have generated the following responses to the question: Can I carry multiple big items so long as only one is equipped?

Response 0
You can carry multiple big items, but only one can be equipped at a time. The "Big" designation for big items does not affect the number of big items a player can carry, but it does affect which traps and curses affect them and prevents thieves from pocketing them.

Response 1
Answer: No, you cannot carry multiple Big items, even if only one is equipped. The rule states that you can carry any number of Small items and only one Big item.

Response 2
Answer: No, you cannot carry multiple Big items as long as only one is equipped. The rules state that you can equip only one Headgear, one suit of Armor, one pair of Footgear, and two "I Hand" Items (or one "2 Hands" Item) unless you have a card that allows you to ignore these limits, such as Hireling or Cheat!, or unless one of the cards says otherwise.

Response 3
No, you cannot carry multiple big items so l

In [54]:
consensus_prompt_template = """Task: Analyze the following responses to the question '{question}' ane rewrite each response so that it is easier to understand. Avoid double negatives. Do alter the meaning or information conveyed in each response.

Responses:
{responses}

Results:
- List the revised responses below using the format 'Response X: [text of the response].' Where X is the response number from above."""


consensus_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", consensus_prompt_template)
    ]
)

def format_responses(responses):
    formatted = [f"Response {i}: {response}" for i, response in enumerate(responses)]
    return "\n".join(formatted)

responses = [
    "No, you cannot carry multiple big items so long as only one is equipped. You can carry multiple big items, but you can only equip one at a time.",
    """You can carry multiple big items, but only one can be equipped at a time. The "Big" designation for big items does not affect the number of big items a player can carry, but it does affect which traps and curses affect them and prevents thieves from pocketing them.""",
    """No, you cannot carry multiple Big items, even if only one is equipped. The rule states that you can carry any number of Small items and only one Big item.""",
    """No, you cannot carry multiple Big items as long as only one is equipped. The rules state that you can equip only one Headgear, one suit of Armor, one pair of Footgear, and two "I Hand" Items (or one "2 Hands" Item) unless you have a card that allows you to ignore these limits, such as Hireling or Cheat!, or unless one of the cards says otherwise.""",
    """No, you cannot carry multiple big items so long as only one is equipped. You can switch Big Items by selling, trading, losing to a Curse or Bad Stuff, or discarding for a Class or Race ability.""",
    """No, you cannot carry multiple Big Items as long as only one is equipped. You can carry any number of Small items, but only one Big one.""",
    """You can carry multiple big items, but you can only equip one at a time.""",
    """No, you cannot carry multiple big items even if only one is equipped. You can only carry one big item at a time.""",
]

from langchain.callbacks.tracers import ConsoleCallbackHandler
import random
# random.shuffle(responses)

result = (consensus_prompt | chat_model.with_config({"callbacks": [ConsoleCallbackHandler()]})
   ).invoke({"question": "Can I carry multiple big items so long as only one is equipped?", "responses": format_responses(responses)})

print(result.content)

[llm/start] [1:llm:HuggingFaceChatModelLocal] Entering LLM run with input:
{
  "prompts": [
    "Human: Task: Analyze the following responses to the question 'Can I carry multiple big items so long as only one is equipped?' ane rewrite each response so that it is easier to understand. Avoid double negatives. Do alter the meaning or information conveyed in each response.\n\nResponses:\nResponse 0: No, you cannot carry multiple big items so long as only one is equipped. You can carry multiple big items, but you can only equip one at a time.\nResponse 1: You can carry multiple big items, but only one can be equipped at a time. The \"Big\" designation for big items does not affect the number of big items a player can carry, but it does affect which traps and curses affect them and prevents thieves from pocketing them.\nResponse 2: No, you cannot carry multiple Big items, even if only one is equipped. The rule states that you can carry any number of Small items and only one Big item.\nRespo

In [86]:
rephrase_prompt_template = """\
I'm looking to improve the clarity of a specific response to a question without changing its original meaning or the factual details it contains. This response can include domain specific information, so it is important to not replace words with synonyms as they may not be synonyms in the domain. The response may involve rules, instructions, or descriptions that need to be conveyed accurately but in a clearer or more straightforward manner. Your task is to reorganize the response to enhance its readability and ensure that it remains unambiguous to the reader. Do not change any terms in the response as they may have special meaning in context, focus solely on making it easier to understand. Here is the question and the response I need rephrased:

Question:
>>>
{question}
>>>

Response:
>>>
{response}
>>>

Remember, the goal is not to add new information or interpret the response, but to reorganize it for clarity while preserving the original intent, terms, and information it conveys.
"""

rephrase_prompt = PromptTemplate(
    template=rephrase_prompt_template,
    input_variables=["resposne", "question"],
)

contradition_prompt_template = """\
I'm looking to know if the following response to the given question contradicts itself. Here is the question and the response I need evaluated:

Question:
>>>
{question}
>>>

Response:
>>>
{response}
>>>

Based on the context, does the response contradict itself? Remember you are determining if the response contradicts itself, not if it contradicts the question asked. Always provide your step-by-step reasoning. After your reasoning, write, on a line all by itself, write "The response contradicts itself" if you believe the response does contradict itself, or "The response does not contradict itself" if you believe the response does not contradict itself."""


contradition_prompt = PromptTemplate(
    template=contradition_prompt_template,
    input_variables=["response", "question"],
)

# sentence = """No, you cannot carry multiple Big items as long as only one is equipped. The rules state that you can equip only one Headgear, one suit of Armor, one pair of Footgear, and two "I Hand" Items (or one "2 Hands" Item) unless you have a card that allows you to ignore these limits, such as Hireling or Cheat!, or unless one of the cards says otherwise."""
question = "Can I carry multiple big items so long as only one is equipped?"
response = "No, you cannot carry multiple Big Items as long as only one is equipped. You can carry any number of Small items, but only one Big one."

print((contradition_prompt | chat_model).invoke({"response": response, "question": question}).content)



Step 1: Analyze the question.
The question asks if it's possible to carry multiple big items while only equipping one.

Step 2: Analyze the response.
The response states that it's not possible to carry multiple Big Items as long as only one is equipped. It also mentions that any number of Small items can be carried, but only one Big one.

Step 3: Compare the question and response.
The question and response are discussing the same topic, which is carrying multiple big items.

Step 4: Determine if the response contradicts itself.
The response clearly states that only one Big Item can be equipped at a time. This is consistent with the rest of the response, which says that any number of Small items can be carried, but only one Big one.

Conclusion:
The response does not contradict itself.


In [24]:
consensus_prompt_template = """\
You are an AI assistant for board games. The following responses have been collected from 7 expert game designers attempting to answer a player submitted question. Determine which responses have the most agreement amonst them. Do not use your own knowledge or opinions to answer the question. Instead, evaluate the responses based on the information provided. We will start with an example followed by the responses/question you are to address.

> Question: If I draw the "Go to Jail" card, can I still collect $200 dollars for passing "Go"?

Response 0: If your pawn passes "Go" on the way to jail, you collect $200. If you land on "Go" after drawing the card, you do not collect $200.

Response 1: No, you cannot collect $200 if you draw the "Go to Jail" card.

Response 2: When you go to jail, you do not pass "Go", so yo do not collect $200.

Response 3: No, you cannot collect $200.

Response 4: No, because you do not pass "Go". On your next turn, you can pay $50 to get out of jail if you roll doubles.

Response 5: No, you cannot collect $200, you can use a "Get out of Jail Free" card or pay $50 to get out of jail on your next turn.

Response 6: If you pass "Go", you may collect $200 based on the rule that players collect $200 for passing "Go".

Evaluate the responses.
Let's think step by step. 
- Responses 1, 2, 3, and 5 all explicitly state you do not collect $200. They make no contradictory claims, though some provide additional information (such as using the "Get out of Jail Free" card).
- Response 4 agrees with 1, 2, 3 and 5 as far as the question is concerned (they all claim the play cannot perform the action asked about), and is consistent with those responses as it does not make any contradictory claims.
- Response 0 and 6 are inconsistent with 1, 2, 3, and 5 as they explicitly claim the player collects $200 when the other responses explicitly claim the player does not collect $200.
- Response 0 and 6 are inconsistent with response 4 as they provide an affirmitive answer (saying the player can perform the action asked about) where as response 4 answers in the negative.
- We can disregard 0 and 6 as they are inconsistent with the majority of responses.
- The remaining responses (1, 2, 3, 4, and 5) are all consistent.
- We will eliminate response 4 as it is the only remaining response that does not explicitly state the player does not collect $200.
- Responses 1, 2, 3, and 5 provide the same answer but some with more information.
- Response 3 is the most consistent as it does not contradict the majority of responses and contains the least extra information.

Therefore, the most consistent response is Response 3.

> Question: {question}

{responses}

Evaluate the responses.
Let's think step by step. \
"""

consensus_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", consensus_prompt_template)
    ]
)

def format_responses(responses):
    formatted = [f"> Response {i}: {response}" for i, response in enumerate(responses)]
    return "\n".join(formatted)

responses = [
    """You can carry multiple big items, but only one can be equipped at a time. The "Big" designation for big items does not affect the number of big items a player can carry, but it does affect which traps and curses affect them and prevents thieves from pocketing them.""",
    """No, you cannot carry multiple Big items, even if only one is equipped. The rule states that you can carry any number of Small items and only one Big item.""",
    """No, you cannot carry multiple Big items as long as only one is equipped. The rules state that you can equip only one Headgear, one suit of Armor, one pair of Footgear, and two "I Hand" Items (or one "2 Hands" Item) unless you have a card that allows you to ignore these limits, such as Hireling or Cheat!, or unless one of the cards says otherwise.""",
    """No, you cannot carry multiple big items so long as only one is equipped. You can switch Big Items by selling, trading, losing to a Curse or Bad Stuff, or discarding for a Class or Race ability.""",
    """No, you cannot carry multiple Big Items as long as only one is equipped. You can carry any number of Small items, but only one Big one.""",
    """You can carry multiple big items, but you can only equip one at a time.""",
    """No, you cannot carry multiple big items even if only one is equipped. You can only carry one big item at a time.""",
]

from langchain.callbacks.tracers import ConsoleCallbackHandler
import random
# random.shuffle(responses)

result = (consensus_prompt | chat_model.with_config({"callbacks": [ConsoleCallbackHandler()]})
   ).invoke({"question": "Can I carry multiple big items so long as only one is equipped?", "responses": format_responses(responses)})

print(result.content)

[llm/start] [1:llm:HuggingFaceChatModelLocal] Entering LLM run with input:
{
  "prompts": [
    "Human: You are an AI assistant for board games. The following responses have been collected from 7 expert game designers attempting to answer a player submitted question. Determine which responses have the most agreement amonst them. Do not use your own knowledge or opinions to answer the question. Instead, evaluate the responses based on the information provided. We will start with an example followed by the responses/question you are to address.\n\n> Question: If I draw the \"Go to Jail\" card, can I still collect $200 dollars for passing \"Go\"?\n\nResponse 0: If your pawn passes \"Go\" on the way to jail, you collect $200. If you land on \"Go\" after drawing the card, you do not collect $200.\n\nResponse 1: No, you cannot collect $200 if you draw the \"Go to Jail\" card.\n\nResponse 2: When you go to jail, you do not pass \"Go\", so yo do not collect $200.\n\nResponse 3: No, you cannot 

In [25]:
from transformers import AutoModel, AutoTokenizer
from sklearn.metrics.pairwise import cosine_similarity

def load_jina_embedding_model() -> AutoModel:
    model_name = 'jinaai/jina-embeddings-v2-base-en'
    model = AutoModel.from_pretrained(
        model_name, 
        trust_remote_code=True, 
        device_map='cpu'
    ).eval()
    if not hasattr(model, 'tokenizer') or model.tokenizer is None:
        model.tokenizer = AutoTokenizer.from_pretrained(
            model_name, 
            trust_remote_code=True, 
            device_map='cpu'
        )
    for parameter in model.parameters():
        parameter.requires_grad = False
    return model

In [26]:
embedding_model = load_jina_embedding_model()

In [27]:
def embed_responses(model, question, responses):
    tokenizer = model.tokenizer

    # Combine each response with the question
    responses = [question + tokenizer.sep_token + response for response in responses]

    return model.encode(responses, normalize_embeddings=True)

In [92]:
import numpy as np
from sklearn.preprocessing import normalize
from sklearn.cluster import KMeans

def normalized_centroid(embeddings):
    # Convert the list of embeddings to a 2D numpy array if it's not already
    embeddings_array = np.array(embeddings)
    
    # Calculate the centroid by computing the mean of the embeddings array
    centroid = np.mean(embeddings_array, axis=0).reshape(1, -1)  # Reshape to 2D array for normalize function
    
    # Use sklearn's normalize function to normalize the centroid
    normalized_centroid = normalize(centroid, norm='l2')
    
    return normalized_centroid.flatten()  # Flatten to convert back to 1D array

def centroid_of_largest_cluster(embeddings):
    # Perform K-means clustering with 2 clusters
    kmeans = KMeans(n_clusters=2, random_state=0).fit(embeddings)
    
    # Get the cluster labels for each embedding
    labels = kmeans.labels_
    
    # Determine the size of each cluster
    cluster_sizes = np.bincount(labels)
    
    # Identify the largest cluster
    largest_cluster = np.argmax(cluster_sizes)
    
    # Return the centroid of the largest cluster
    largest_cluster_centroid = kmeans.cluster_centers_[largest_cluster]

    # Normalize the centroid
    normalized_centroid = normalize(largest_cluster_centroid.reshape(1, -1), norm='l2')
    
    return normalized_centroid

def find_vector_elimiate_furthest(embeddings):
    indices_to_exclude = []
    orig_embeddings = embeddings
    centroid = embeddings[0]
    while len(orig_embeddings) - len(indices_to_exclude) > 2:
        indices_remaining = [i for i in range(len(orig_embeddings)) if i not in indices_to_exclude]
        print(indices_remaining)
        embeddings = [embedding for i, embedding in enumerate(orig_embeddings) if i not in indices_to_exclude]
        centroid = normalized_centroid(embeddings)
        sorted_indices = sort_embedding_indices(embeddings, centroid)
        furthest_index = sorted_indices[-1]
        # Remove furthest_index from embeddings, keeping in mind embeddings
        # might be a numpy array
        if isinstance(embeddings, np.ndarray):
            embeddings = np.delete(embeddings, furthest_index, axis=0)
        else:
            embeddings.pop(furthest_index)
        indices_to_exclude.append(indices_remaining[furthest_index])
        indices_remaining = [i for i in range(len(orig_embeddings)) if i not in indices_to_exclude]
        print(indices_remaining)
    return centroid
        

def sort_embedding_indices(embeddings, query_vector):
    # Ensure the query_vector is a 2D array for cosine_similarity function
    query_vector_2d = query_vector.reshape(1, -1)
    
    # Calculate cosine similarities between the query_vector and all embeddings
    similarities = cosine_similarity(query_vector_2d, embeddings)
    
    # Sort the indices of the embeddings based on their similarity to the query_vector
    sorted_indices = np.argsort(similarities.flatten())[::-1]

    return sorted_indices

def rank_responses_by_majority_consensus(embedding_model, question, responses):
    # Create the embeddings for the responses
    embeddings = embed_responses(embedding_model, question, responses)
    
    # Find the centroid of the embeddings
    # centroid = normalized_centroid(embeddings)
    centroid = centroid_of_largest_cluster(embeddings)
    # centroid = find_vector_elimiate_furthest(embeddings)

    # Pray there are more correct responses than wrong ones so that the centroid
    # is closer to a correct answer than a wrong one
    sorted_indices = sort_embedding_indices(embeddings, centroid)

    # Return the response closest to the centroid
    return [responses[i] for i in sorted_indices]

In [93]:
test_cases = [
    {
        "question": """If I draw the "Go to Jail" card, can I still collect $200 dollars for passing "Go"?""",
        "responses": [
            "If your pawn passes \"Go\" on the way to jail, you collect $200. If you land on \"Go\" after drawing the card, you do not collect $200.",
            "No, you cannot collect $200 if you draw the \"Go to Jail\" card.",
            "When you go to jail, you do not pass \"Go\", so yo do not collect $200.",
            "No, you cannot collect $200.",
            "No, because you do not pass \"Go\". On your next turn, you can pay $50 to get out of jail if you roll doubles.",
            "No, you cannot collect $200, you can use a \"Get out of Jail Free\" card or pay $50 to get out of jail on your next turn.",
            "If you pass \"Go\", you may collect $200 based on the rule that players collect $200 for passing \"Go\"."
        ],
    },
    {
        "question": "Can I carry multiple big items so long as only one is equipped?",
        "responses": [
            """You can carry multiple big items, but only one can be equipped at a time. The "Big" designation for big items does not affect the number of big items a player can carry, but it does affect which traps and curses affect them and prevents thieves from pocketing them.""",
            """No, you cannot carry multiple Big items, even if only one is equipped. The rule states that you can carry any number of Small items and only one Big item.""",
            """No, you cannot carry multiple Big items as long as only one is equipped. The rules state that you can equip only one Headgear, one suit of Armor, one pair of Footgear, and two "I Hand" Items (or one "2 Hands" Item) unless you have a card that allows you to ignore these limits, such as Hireling or Cheat!, or unless one of the cards says otherwise.""",
            """No, you cannot carry multiple big items so long as only one is equipped. You can switch Big Items by selling, trading, losing to a Curse or Bad Stuff, or discarding for a Class or Race ability.""",
            """No, you cannot carry multiple Big Items as long as only one is equipped. You can carry any number of Small items, but only one Big one.""",
            """You can carry multiple big items, but you can only equip one at a time.""",
            """No, you cannot carry multiple big items even if only one is equipped. You can only carry one big item at a time.""",
            """No, you cannot carry multiple big items as long as only one is equipped. You can carray multiple big items, but only one can be equipped."""
        ]
    },
    {
        "question": "Can I carry multiple big items so long as only one is equipped?",
        "responses": [
            "No, you cannot carry multiple big items so long as only one is equipped. You can carry multiple big items, but you can only equip one at a time.",
            "No, you cannot carry multiple big items so long as only one is equipped. The rules state that you can carry any number of small items, but only one big item at a time. Additionally, you cannot alter the status of your items during combat or while running away, which implies that you cannot equip or unequip items during these phases of the game.",
            "No, you cannot carry multiple big items so long as only one is equipped.",
            "No, a player cannot carry multiple big items, even if only one is equipped. They can only carry one big item at a time.",
            "Yes, a player can carry multiple big items, but only one can be equipped at a time.",
            "Yes, you can carry multiple big items as long as only one is equipped at a time.",
            "No, you cannot carry multiple big items so long as only one is equipped. The rules state that you can carry any number of small items, but only one big item at a time. Additionally, you cannot alter the status of your items during combat or while running away, which implies that you cannot equip or unequip items during these phases of the game.",
        ]
    }
]

for test_case in test_cases:
    question = test_case["question"]
    responses = test_case["responses"]
    print(f"Question: {question}")
    for response in rank_responses_by_majority_consensus(embedding_model, question, responses):
        print(response)
    print()
    print()

Question: If I draw the "Go to Jail" card, can I still collect $200 dollars for passing "Go"?
[0, 1, 2, 3, 4, 5, 6]
[0, 1, 2, 3, 5, 6]
[0, 1, 2, 3, 5, 6]
[1, 2, 3, 5, 6]
[1, 2, 3, 5, 6]
[1, 2, 3, 5]
[1, 2, 3, 5]
[1, 2, 3]
[1, 2, 3]
[1, 3]
No, you cannot collect $200 if you draw the "Go to Jail" card.
No, you cannot collect $200.
When you go to jail, you do not pass "Go", so yo do not collect $200.
No, you cannot collect $200, you can use a "Get out of Jail Free" card or pay $50 to get out of jail on your next turn.
If you pass "Go", you may collect $200 based on the rule that players collect $200 for passing "Go".
If your pawn passes "Go" on the way to jail, you collect $200. If you land on "Go" after drawing the card, you do not collect $200.
No, because you do not pass "Go". On your next turn, you can pay $50 to get out of jail if you roll doubles.


Question: Can I carry multiple big items so long as only one is equipped?
[0, 1, 2, 3, 4, 5, 6, 7]
[0, 1, 3, 4, 5, 6, 7]
[0, 1, 3, 4, 5

In [29]:
# reword_consensus_prompt_template = """\
# You are an AI assistant for board games. The following responses have been collected from 7 expert game designers attempting to answer a player submitted question. Reword each response for clarity. Do not change the meaning of the information conveyed. Listed the reworded responses in the same format as the original responses.

# > Example.

# > Question: If I draw the "Go to Jail" card, can I still collect $200 dollars for passing "Go"?

# Response 0: If your pawn passes "Go" on the way to jail, you collect $200. If you land on "Go" after drawing the card, you do not collect $200.

# Response 1: No, you cannot collect $200 if you draw the "Go to Jail" card.

# Response 2: When you go to jail, you do not pass "Go", so yo do not collect $200.

# Response 3: No, you cannot collect $200.

# Response 4: No, because you do not pass "Go". On your next turn, you can pay $50 to get out of jail if you roll doubles.

# Response 5: No, you cannot collect $200, you can use a "Get out of Jail Free" card or pay $50 to get out of jail on your next turn.

# Response 6: If you pass "Go" you may collect $200. This is based on the rule that players collect $200 for passing "Go".

# > Reword the responses for clarity:

# Response 0: If your pawn passes "Go" while moving towards the jail, you collect $200. However, if you land directly on "Go" after drawing the "Go to Jail" card, you do not collect $200.

# Response 1: No, you cannot collect $200 if you draw the "Go to Jail" card.

# Response 2: No, when you go to jail, you do not collect $200 because you do not pass "Go".

# Response 3: No, you cannot collect $200.

# Response 4: No, because you do not pass "Go". On your next turn, you can pay $50 to get out of jail if you roll doubles.

# Response 5: No, you cannot collect $200, you can use a "Get out of Jail Free" card or pay $50 to get out of jail on your next turn.

# Response 6: If you pass "Go", you may collect $200 based on the rule that players collect $200 for passing "Go".

# > Actual question and responses.

# > Question: {question}

# {responses}

# """

# reword_consensus_prompt = ChatPromptTemplate.from_messages(
#     [
#         ("human", consensus_prompt_template)
#     ]
# )


# def format_responses(responses):
#     formatted = [f"> Response {i}: {response}" for i, response in enumerate(responses)]
#     return "\n".join(formatted)

# responses = [
#     """You can carry multiple big items, but only one can be equipped at a time. The "Big" designation for big items does not affect the number of big items a player can carry, but it does affect which traps and curses affect them and prevents thieves from pocketing them.""",
#     """No, you cannot carry multiple Big items, even if only one is equipped. The rule states that you can carry any number of Small items and only one Big item.""",
#     """No, you cannot carry multiple Big items as long as only one is equipped. The rules state that you can equip only one Headgear, one suit of Armor, one pair of Footgear, and two "I Hand" Items (or one "2 Hands" Item) unless you have a card that allows you to ignore these limits, such as Hireling or Cheat!, or unless one of the cards says otherwise.""",
#     """No, you cannot carry multiple big items so long as only one is equipped. You can switch Big Items by selling, trading, losing to a Curse or Bad Stuff, or discarding for a Class or Race ability.""",
#     """No, you cannot carry multiple Big Items as long as only one is equipped. You can carry any number of Small items, but only one Big one.""",
#     """You can carry multiple big items, but you can only equip one at a time.""",
#     """No, you cannot carry multiple big items even if only one is equipped. You can only carry one big item at a time.""",
# ]

# from langchain.callbacks.tracers import ConsoleCallbackHandler
# import random
# # random.shuffle(responses)

# result = (consensus_prompt | chat_model.with_config({"callbacks": [ConsoleCallbackHandler()]})
#    ).invoke({"question": "Can I carry multiple big items so long as only one is equipped?", "responses": format_responses(responses)})

# print(result.content)

[llm/start] [1:llm:HuggingFaceChatModelLocal] Entering LLM run with input:
{
  "prompts": [
    "Human: You are an AI assistant for board games. The following responses have been collected from 7 expert game designers attempting to answer a player submitted question. Reword each response for clarity. Do not change the meaning of the information conveyed. Listed the reworded responses in the same format as the original responses.\n\n> Example.\n\n> Question: If I draw the \"Go to Jail\" card, can I still collect $200 dollars for passing \"Go\"?\n\nResponse 0: If your pawn passes \"Go\" on the way to jail, you collect $200. If you land on \"Go\" after drawing the card, you do not collect $200.\n\nResponse 1: No, you cannot collect $200 if you draw the \"Go to Jail\" card.\n\nResponse 2: When you go to jail, you do not pass \"Go\", so yo do not collect $200.\n\nResponse 3: No, you cannot collect $200.\n\nResponse 4: No, because you do not pass \"Go\". On your next turn, you can pay $50 to